## 15.1 数据插入
&emsp;&emsp;INSERT 用来将行插入（或添加）到数据库表。插入有几种方式：
- 插入完整的行；
- 插入行的一部分；
- 插入某些查询的结果。

>**提示**：使用INSERT 语句可能需要客户端/服务器DBMS 中的特定安全权限。在你试图使用INSERT 前，应该保证自己有足够的安全权限。

### 15.1.1 插入完整的行
&emsp;&emsp;把数据插入表中的最简单方法是使用基本的INSERT 语法，它要求指定表名和插入到新行中的值：

In [1]:
import sqlite3 as sql 
import pandas as pd

# 构造插入函数
def insert_sql(insert):
    conn = sql.connect('tysql.sqlite')
    cur = conn.cursor()
    cur.execute(insert);
    cur.close()
    conn.commit()
    conn.close()
    print('Done!')

In [3]:
# 将一个新顾客插入到Customers 表
insert = '''
INSERT INTO Customers
VALUES('1000000006',
        'Toy Land',
        '123 Any Street',
        'New York',
        'NY',
        '11111',
        'USA',
        NULL,
        NULL);
'''
insert_sql(insert)

Done!


>**提示**：在某些SQL 实现中，跟在INSERT 之后的INTO 关键字是可选的。但是，即使不一定需要，最好还是提供这个关键字，这样做将保证SQL代码在DBMS 之间可移植。

``` python
# 编写INSERT 语句的更安全（不过更烦琐）的方法
insert = '''
INSERT INTO Customers(cust_id,
        cust_name,
        cust_address,
        cust_city,
        cust_state,
        cust_zip,
        cust_country,
        cust_contact,
        cust_email)
VALUES('1000000006',
        'Toy Land',
        '123 Any Street',
        'New York',
        'NY',
        '11111',
        'USA',
        NULL,
        NULL);
'''
insert_sql(insert)
```

>**提示**：不要使用没有明确给出列的INSERT 语句。给出列能使SQL 代码继续发挥作用，即使表结构发生了变化。

>**注意**：不管使用哪种INSERT 语法，VALUES 的数目都必须正确。如果不提供列名，则必须给每个表列提供一个值；如果提供列名，则必须给列出的每个列一个值。否则，就会产生一条错误消息，相应的行不能成功插入。

### 15.1.2 插入部分行
&emsp;&emsp;使用INSERT 的推荐方法是明确给出表的列名。使用这种语法，还可以省略列，这表示可以只给某些列提供值，给其他列不提供值。

``` python
# 省略部分列
insert = '''
INSERT INTO Customers(cust_id,
        cust_name,
        cust_address,
        cust_city,
        cust_state,
        cust_zip,
        cust_country)
VALUES('1000000006',
        'Toy Land',
        '123 Any Street',
        'New York',
        'NY',
        '11111',
        'USA');
'''
insert_sql(insert)
```

>**注意**：如果表的定义允许，则可以在INSERT 操作中省略某些列。省略的列必须满足以下某个条件：
- 该列定义为允许NULL 值（无值或空值）；
- 在表定义中给出默认值。这表示如果不给出值，将使用默认值。

>**警告**：如果表中不允许有NULL 值或者默认值，这时却省略了表中的值，DBMS 就会产生错误消息，相应的行不能成功插入。

### 15.1.3 插入检索出的数据
&emsp;&emsp;INSERT 一般用来给表插入具有指定列值的行。INSERT 还存在另一种形式，可以利用它将SELECT 语句的结果插入表中，这就是所谓的INSERT SELECT：

``` python
# 把另一表中的顾客列合并到Customers 表中
insert = '''
INSERT INTO Customers(cust_id,
        cust_contact,
        cust_email,
        cust_name,
        cust_address,
        cust_city,
        cust_state,
        cust_zip,
        cust_country)
SELECT cust_id,
        cust_contact,
        cust_email,
        cust_name,
        cust_address,
        cust_city,
        cust_state,
        cust_zip,
        cust_country
FROM CustNew;
'''
insert_sql(insert)
```

>**提示**：为简单起见，这个例子在INSERT 和SELECT 语句中使用了相同的列名。但是，不一定要求列名匹配。事实上，DBMS 一点儿也不关心SELECT返回的列名。它使用的是列的位置，因此SELECT 中的第一列（不管其列名）将用来填充表列中指定的第一列，第二列将用来填充表列中指定的第二列，如此等等。

>**说明**：INSERT 通常只插入一行。要插入多行，必须执行多个INSERT 语句。INSERT SELECT 是个例外，它可以用一条INSERT 插入多行，不管SELECT语句返回多少行，都将被INSERT 插入。

## 15.2 从一个表复制到另一个表
&emsp;&emsp;有一种数据插入不使用INSERT 语句。要将一个表的内容复制到一个全新的表（运行中创建的表），可以使用SELECT INTO 语句。

>**说明**：DB2 不支持这里描述的SELECT INTO。

&emsp;&emsp;有一种数据插入不使用INSERT 语句。要将一个表的内容复制到一个全新的表（运行中创建的表），可以使用SELECT INTO 语句。

>**说明**：INSERT SELECT 与SELECT INTO之间的一个重要差别是前者插入数据，而后者导出数据。

``` python
# 使用SELECT INTO 将Customers 表复制到CustCopy 表
insert = '''
SELECT *
INTO CustCopy
FROM Customers;
'''
insert_sql(insert)
```

In [3]:
# MariaDB、MySQL、Oracle、PostgreSQL 和SQLite 使用的语法
insert = '''
CREATE TABLE CustCopy AS
SELECT * FROM Customers;
'''
insert_sql(insert)

Done!


&emsp;&emsp;在使用SELECT INTO 时，需要知道一些事情：
- 任何SELECT 选项和子句都可以使用，包括WHERE 和GROUP BY；
- 可利用联结从多个表插入数据；
- 不管从多少个表中检索数据，数据都只能插入到一个表中。

>**提示**：SELECT INTO 是试验新SQL 语句前进行表复制的很好工具。先进行复制，可在复制的数据上测试SQL 代码，而不会影响实际的数据。

## 15.3 小结
&emsp;&emsp;本章学习了学习了使用INSERT 的几种方法，为什么要明确使用列名，如何用INSERT SELECT 从其他表中导入行，如何用SELECT INTO 将行导出到一个新表。